In [4]:
import numpy as np
import pandas as pd
import imutils
import os
import os.path
import glob
from skimage import data, io
import warnings
warnings.filterwarnings("ignore")

photo_image_folder: 訓練用的電話號碼圖片
output_dir: 切割後的數字圖片，將不同的數字放到不同的folder內

In [14]:
phone_image_folder = 'sample_numbers'
output_dir = 'extract_number_images'
phone_image_files = glob.glob(os.path.join(phone_image_folder, '*'))

因為電話號碼中有'-'符號，為了去掉'-'號，將平均值大於233.75的欄位標示成空白欄

In [6]:
def Check(data):
    if (data.mean() >= 233.75 ):
        return 0
    else:
        return 1

將每個數字後面加上一個空白的pixel，並回傳修改後的數字list

In [7]:
def AddPixels(data, width):
    pixels = []
    for num in range(0, width):
        d = data[:][num]
        is_valid = Check(d)
        if (is_valid == 1):
            pixels.append(num)
    return pixels

因為有些電話號碼中有分機號碼(#)，用空白欄位切割方式不適用，所以用分割後的畫素長度進行判斷，如果畫數長度超過10pixel就切成兩個字

In [8]:
def CheckSize(number):
    chars = {}
    if (len(number) > 10):
        char1 = [] 
        char2 = []
        for n in range(0, len(number)):
            if n < 8:
                char1.append(number[n])
            else:
                char2.append(number[n])
        chars[1] = char1
        chars[2] = char2
    else:
        chars[1] = number
    return chars

將圖片從空白處進行分割，並將每個數字所使用的dataframe存到list內

In [11]:
def GetChars(pixels):
    chars = []
    number = []
    for num in pixels:
        index = pixels.index(num)
        if(index != len(pixels)-1):
            if (pixels[index + 1] - num == 1 ):
                number.append(num)
            else:
                number.append(num)
                if(len(number) > 10):
                    char1 = []
                    char2 = []
                    for n in range(0, len(number)):
                        if (n< 8):
                            char1.append(number[n])
                        else:
                            char2.append(number[n])
                    chars.append(tuple(char1))
                    chars.append(tuple(char2))
                else:
                    chars.append(tuple(number))
                number.clear()
        else:
            chars.append(tuple(number))
    return chars

In [15]:
counts = {}
for (i, phone_image_file) in enumerate(phone_image_files):
    filename = os.path.basename(phone_image_file)
    photo_correct_text = os.path.splitext(filename)[0]
    text = []
    # 將檔名進行分割，利用檔名建立label，提供後續訓練模型使用
    for t in photo_correct_text:
        if(len(t) > 0 ):
            text.append(t)
    print(text)
    image = data.imread(phone_image_file)
    image_pd = pd.DataFrame(image)
    width = image_pd.shape[1]
    pixels = AddPixels(image_pd, width)
    empty = image_pd[width-1]
    chars = GetChars(pixels)
    for char in chars:
        ch = list(char)
        char_pd = image_pd[ch]
        name = text[chars.index(char)]
        # 將每個畫數不是白色(255)就改成黑色(0)，在訓練模型時可以得到最佳效果
        for c in ch:
            char_pd[c] = char_pd[c].apply(lambda x : 0 if x < 255 else 255)
        char_pd[len(ch)] = empty
        save_path = os.path.join(output_dir, name)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        # 將分割後的檔案依據label存到不同目錄
        count = counts.get(name, 1)
        p = os.path.join(save_path, '{}.png'.format(str(count).zfill(4)))
        io.imsave(p, char_pd)
        counts[name] = count + 1

['0', '2', '2', '1', '0', '0', '2', '8', '6', '2']
['0', '2', '2', '1', '7', '5', '5', '5', '8', '8']
['0', '2', '2', '1', '8', '1', '9', '9', '9', '9', '#', '3', '2', '6', '9']
['0', '2', '2', '1', '8', '1', '9', '9', '9', '9']
['0', '2', '2', '2', '0', '3', '7', '2', '3', '6']
['0', '2', '2', '2', '0', '8', '1', '8', '8', '8']
['0', '2', '2', '2', '1', '8', '2', '0', '1', '8']
['0', '2', '2', '2', '2', '3', '5', '6', '5', '6']
['0', '2', '2', '2', '2', '8', '8', '8', '4', '4']
['0', '2', '2', '2', '3', '1', '1', '1', '1', '1']
['0', '2', '2', '2', '4', '8', '2', '7', '6', '6']
['0', '2', '2', '2', '5', '3', '2', '0', '8', '7']
['0', '2', '2', '2', '5', '4', '4', '4', '5', '3']
['0', '2', '2', '2', '5', '4', '7', '7', '7', '1']
['0', '2', '2', '2', '5', '5', '7', '7', '3', '3']
['0', '2', '2', '2', '5', '5', '9', '3', '8', '6']
['0', '2', '2', '2', '5', '7', '0', '7', '7', '5']
['0', '2', '2', '2', '5', '7', '6', '4', '4', '5']
['0', '2', '2', '2', '5', '8', '6', '9', '0', '6']
['0', 

['0', '2', '2', '5', '5', '6', '5', '9', '0', '8']
['0', '2', '2', '5', '5', '8', '0', '0', '9', '6']
['0', '2', '2', '5', '5', '8', '6', '7', '5', '8']
['0', '2', '2', '5', '5', '9', '3', '1', '5', '9']
['0', '2', '2', '5', '5', '9', '6', '5', '1', '0']
['0', '2', '2', '5', '5', '9', '7', '3', '9', '1']
['0', '2', '2', '5', '6', '1', '7', '8', '5', '9']
['0', '2', '2', '5', '6', '2', '0', '9', '3', '1']
['0', '2', '2', '5', '6', '2', '3', '3', '6', '0']
['0', '2', '2', '5', '6', '2', '9', '2', '2', '2']
['0', '2', '2', '5', '6', '3', '5', '9', '9', '0']
['0', '2', '2', '5', '6', '3', '6', '2', '3', '9']
['0', '2', '2', '5', '6', '3', '8', '3', '5', '6']
['0', '2', '2', '5', '6', '3', '9', '2', '9', '3']
['0', '2', '2', '5', '6', '4', '2', '0', '2', '0']
['0', '2', '2', '5', '6', '7', '0', '6', '8', '7']
['0', '2', '2', '5', '6', '7', '2', '7', '7', '0']
['0', '2', '2', '5', '6', '7', '3', '7', '8', '7']
['0', '2', '2', '5', '6', '7', '8', '7', '6', '9']
['0', '2', '2', '5', '6', '7', 

['0', '2', '2', '7', '5', '5', '0', '5', '6', '7']
['0', '2', '2', '7', '5', '5', '5', '2', '0', '0']
['0', '2', '2', '7', '5', '8', '0', '9', '0', '9']
['0', '2', '2', '7', '5', '8', '1', '3', '1', '4']
['0', '2', '2', '7', '5', '8', '2', '6', '0', '9']
['0', '2', '2', '7', '5', '8', '8', '8', '1', '2']
['0', '2', '2', '7', '5', '9', '3', '5', '6', '8']
['0', '2', '2', '7', '6', '1', '5', '5', '1', '0']
['0', '2', '2', '7', '6', '3', '7', '0', '0', '2']
['0', '2', '2', '7', '6', '4', '4', '7', '8', '8']
['0', '2', '2', '7', '6', '4', '8', '1', '8', '1']
['0', '2', '2', '7', '6', '5', '5', '8', '5', '9']
['0', '2', '2', '7', '6', '5', '7', '6', '8', '8']
['0', '2', '2', '7', '6', '6', '9', '6', '6', '3']
['0', '2', '2', '7', '6', '7', '3', '9', '4', '6']
['0', '2', '2', '7', '7', '1', '0', '3', '0', '8']
['0', '2', '2', '7', '7', '1', '1', '6', '1', '6']
['0', '2', '2', '7', '7', '1', '1', '8', '0', '8']
['0', '2', '2', '7', '7', '1', '2', '9', '7', '7']
['0', '2', '2', '7', '7', '1', 

['0', '2', '8', '7', '7', '2', '3', '6', '8', '8']
['0', '2', '8', '7', '7', '2', '4', '6', '1', '8']
['0', '2', '8', '7', '7', '2', '5', '0', '8', '9']
['0', '2', '8', '7', '7', '2', '5', '7', '2', '8']
['0', '2', '8', '7', '7', '2', '8', '8', '0', '0', '#', '2', '3', '1', '1']
['0', '2', '8', '7', '7', '2', '8', '8', '3', '2']
['0', '2', '8', '7', '7', '2', '9', '6', '6', '6']
['0', '2', '8', '7', '7', '3', '2', '1', '3', '6']
['0', '2', '8', '7', '7', '3', '7', '1', '0', '3']
['0', '2', '8', '7', '7', '3', '7', '8', '9', '9']
['0', '2', '8', '7', '7', '3', '8', '3', '3', '5']
['0', '2', '8', '7', '7', '3', '9', '0', '5', '0']
['0', '2', '8', '7', '7', '3', '9', '1', '9', '0']
['0', '2', '8', '7', '7', '3', '9', '2', '3', '8']
['0', '2', '8', '7', '7', '3', '9', '2', '9', '2']
['0', '2', '8', '7', '8', '0', '1', '3', '6', '6']
['0', '2', '8', '7', '8', '6', '0', '0', '2', '9']
['0', '2', '8', '7', '8', '6', '7', '5', '8', '8']
['0', '2', '8', '7', '8', '7', '8', '1', '8', '6']
['0', 

['0', '4', '2', '4', '5', '2', '5', '2', '7', '5']
['0', '4', '2', '4', '6', '1', '4', '1', '2', '8']
['0', '4', '2', '4', '7', '2', '3', '4', '5', '6']
['0', '4', '2', '5', '1', '3', '1', '1', '8', '8']
['0', '4', '2', '5', '3', '5', '7', '3', '7', '3']
['0', '4', '2', '5', '6', '9', '2', '7', '9', '8']
['0', '4', '2', '5', '8', '2', '3', '5', '6', '8']
['0', '4', '2', '5', '9', '3', '1', '2', '2', '3']
['0', '4', '2', '6', '5', '2', '8', '3', '0', '0']
['0', '4', '2', '6', '8', '8', '3', '6', '7', '7']
['0', '4', '2', '7', '0', '8', '2', '2', '6', '8']
['0', '4', '2', '7', '0', '8', '5', '0', '8', '0']
['0', '4', '7', '2', '2', '9', '5', '1', '7']
['0', '4', '7', '9', '7', '8', '0', '0', '0']
['0', '4', '8', '5', '2', '8', '2', '9', '7']
['0', '4', '9', '2', '2', '6', '1', '1', '2', '3']
['0', '4', '9', '2', '2', '9', '2', '7', '6', '7', '#', '0']
['0', '4', '9', '2', '8', '0', '2', '9', '8', '8']
['0', '4', '9', '2', '9', '0', '1', '6', '4', '9']
['0', '4', '9', '2', '9', '8', '4', 